In [1]:
import codecs, glob, os
import numpy as np
import pandas as pd
from sklearn import metrics

import paddle.nn.functional as F
import paddle
import paddle.nn as nn
from paddle.io import DataLoader, Dataset
import paddle.optimizer as optim
from paddlenlp.data import Pad

import scipy.io as sio

In [2]:
path = './'
df = pd.read_csv(path+'trainreference.csv')
class myDataset(Dataset):
    def __init__(self, df, idx=None, if_train=True):
        self.if_train = if_train
        if self.if_train:
            self.paths = df.loc[idx, 'name'].reset_index(drop=True)
            self.labels = df.loc[idx, 'tag'].reset_index(drop=True)
        else:
            self.paths = df['name'].reset_index(drop=True)
            self.labels = df['tag'].reset_index(drop=True)

    def __getitem__(self, index):
        if self.if_train:
            sample = sio.loadmat(path+'train/'+self.paths[index])['ecgdata']
        else:
            sample = sio.loadmat(path+'val/'+self.paths[index])['ecgdata']
        return sample, self.labels[index]

    def __len__(self):
        return len(self.paths)

In [3]:
class SeqNet(nn.Layer):
    def __init__(self):
        super(SeqNet, self).__init__()
        # input 
        self.conv1 = nn.Conv1D(12, 10, 50)
        self.conv2 = nn.Conv1D(12, 10, 200)
        self.conv3 = nn.Conv1D(12, 10, 500)
        self.conv4 = nn.Conv1D(12, 10, 1000)
        # self.pooling = nn.MaxPool2D((1, 200))
        self.pooling = nn.MaxPool1D(200)
        self.fc1 = nn.Linear(900, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        batch_size = x.shape[0]
        
        out1 = self.pooling(F.relu(self.conv1(x)))
        out2 = self.pooling(F.relu(self.conv2(x)))
        out3 = self.pooling(F.relu(self.conv3(x)))
        out4 = self.pooling(F.relu(self.conv4(x)))

        # # out = torch.cat([out1, out2, out3, out4], 2)  
        out = paddle.concat([out1, out2, out3, out4], 2)
        # out = out.view(batch_size, -1)
        out = paddle.reshape(out, [batch_size, -1])
        out = self.fc1(out)
        out = F.relu(out)
        out = F.dropout(out, p=0.6)
        out = self.fc2(out)

        return out

In [4]:
import time
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
max_epoch = 200
batch_size = 32
model_save_dir = path+'model/'
def train_model(model, criterion, optimizer, lr_scheduler=None):
    total_iters=len(trainloader)
    print('--------------total_iters:{}'.format(total_iters))
    since = time.time()
    best_loss = 1e7
    best_epoch = 0
    best_f1 = 0
    #
    iters = len(trainloader)
    for epoch in range(1,max_epoch+1):
        model.train()
        begin_time=time.time()
        # print('learning rate:{}'.format(optimizer.param_groups[-1]['lr']))
        # print('Fold{} Epoch {}/{}'.format(fold+1,epoch, max_epoch))
        running_corrects_linear = 0
        count=0
        train_loss = []
        for i, (inputs, labels) in (enumerate(trainloader)):
            # print(inputs)
            count+=1
            # inputs = inputs.to(device)
            inputs = inputs.cuda()
            # labels = labels.float().to(device)
            labels = labels.cuda()
            # labels = paddle.reshape(labels, (30, 1))
            labels = paddle.cast(labels, dtype='float32')

            out_linear = model(inputs)
            out_linear = paddle.reshape(out_linear, (batch_size,))
            loss = criterion(out_linear, labels)
            # loss = criterion(out_linear, labels.unsqueeze(1))

            # optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            optimizer.clear_grad()
            # 更新cosine学习率
            if lr_scheduler!=None:
                lr_scheduler.step(epoch + count / iters)
            if print_interval>0 and (i % print_interval == 0 or out_linear.size()[0] < train_batch_size):
                spend_time = time.time() - begin_time
                if epoch % 50 == 0:
                    print(
                    ' Fold:{} Epoch:{}({}/{}) loss:{:.3f} lr:{:.7f} epoch_Time:{}min:'.format(
                        fold+1,epoch, count, total_iters,
                        loss.item(), optimizer.param_groups[-1]['lr'],
                        spend_time / count * total_iters // 60 - spend_time // 60))
            #
            train_loss.append(loss.item())
        #lr_scheduler.step()
        val_f1, val_loss= val_model(model, criterion)
        if epoch % 50 == 0:
            print('valf1: {:.4f}  valLogLoss: {:.4f}'.format(val_f1, val_loss))
        model_out_path = model_save_dir+"/"+'fold_'+str(fold+1)+'_'+str(epoch) + '.pth'
        best_model_out_path = model_save_dir+"/"+'fold_'+str(fold+1)+'_best'+'.pth'
        #save the best model
        if val_f1 >= best_f1:
            best_loss = val_loss
            best_f1 = val_f1
            best_epoch=epoch
            paddle.save(model.state_dict(), best_model_out_path)
            if epoch % 50 == 0:
                print("save best epoch: {} best f1: {:.5f} best logloss: {:.5f}".format(best_epoch,val_f1,val_loss))
  
    print('Fold{} Best f1: {:.3f} Best logloss: {:.3f} Best epoch:{}'.format(fold+1,best_f1, best_loss,best_epoch))
    time_elapsed = time.time() - since
    return best_loss, best_f1

@paddle.no_grad()
def val_model(model, criterion):
    model.eval()
    running_loss = 0.0
    running_corrects = 0
    cont = 0
    outPre = []
    outLabel = []
    pres_list=[]
    labels_list=[]
    for data in val_loader:
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        outputs = model(inputs)
        # pres_list+=outputs.sigmoid().detach().cpu().numpy().tolist()
        pres_list+=paddle.nn.functional.sigmoid(outputs).detach().cpu().numpy().tolist()
        labels_list+=labels.detach().cpu().numpy().tolist()

    preds = np.array(pres_list)
    labels = np.array(labels_list)
    val_f1 = metrics.f1_score(labels, list(map(lambda x: 1 if x > 0.5 else 0, preds)))
    log_loss = metrics.log_loss(labels, preds)#
    return val_f1, log_loss

In [5]:
# def setup_seed(seed):
#      torch.manual_seed(seed)
#      torch.cuda.manual_seed_all(seed)
#      np.random.seed(seed)
#      random.seed(seed)
#      torch.backends.cudnn.deterministic = True
# setup_seed(2021)
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=10)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = paddle.device.get_device()
print(device)

gpu:0


In [90]:
# Train_Loader = DataLoader(
#                 myDataset(df, [i for i in range(1200)]), 
#                 batch_size=30, shuffle=True)
# for i in Train_Loader:
#     x1 = i[0]
#     y1 = i[1]
#     x1 = x1.cuda()
#     y1 = y1.cuda()
    
#     break
# x1.shape
# x1


In [91]:
# model = SeqNet()
# model.to(device)
# out = model(x1)
# out = paddle.reshape(out, (30,))
# print(out)
# # y1 = paddle.reshape(y1, (30, 1))
# y1 = paddle.cast(y1, dtype='float32')
# print(y1)


# # loss = paddle.mean(out)
# criterion = nn.BCEWithLogitsLoss()
# loss = criterion(out, y1)
# print(loss)
# adam = paddle.optimizer.AdamW(weight_decay=0.01, learning_rate=0.1,
#         parameters=model.parameters())
# out.backward()
# adam.step()
# adam.clear_grad()

In [ ]:
criterion = nn.BCEWithLogitsLoss()
print_interval=-1
kfold_best_loss = []
kfold_best_f1 = []
# print(len(df))
for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['tag'].values)):
    trainloader = DataLoader(
        myDataset(df, train_idx), 
        batch_size=32, shuffle=True, num_workers=1)
    val_loader = DataLoader(
        myDataset(df, val_idx), 
        batch_size=128, shuffle=False, num_workers=1)
    model = SeqNet()
    model.to(device)
    optimizer = optim.AdamW(learning_rate=1e-4, parameters=model.parameters(),weight_decay=5e-4)
    # # optimizer = optim.Adam(parameters=model.parameters(), learning_rate=1e-4)
    scheduler = optim.lr.CosineAnnealingDecay(1e-4, T_max=max_epoch)

    best_loss, best_acc = train_model(model, criterion, optimizer, lr_scheduler=scheduler)
    kfold_best_loss.append(best_loss)
    kfold_best_f1.append(best_acc)

print(kfold_best_f1)                  
print('loss...', np.mean(kfold_best_loss), 'f1...', np.mean(kfold_best_f1))

W1129 15:34:26.066895 11426 device_context.cc:404] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W1129 15:34:26.071918 11426 device_context.cc:422] device: 0, cuDNN Version: 7.6.


--------------total_iters:45
valf1: 0.8163  valLogLoss: 0.4785
valf1: 0.8344  valLogLoss: 0.7376
valf1: 0.8313  valLogLoss: 0.9174
valf1: 0.7925  valLogLoss: 1.1313
Fold1 Best f1: 0.859 Best logloss: 0.755 Best epoch:69
--------------total_iters:45
valf1: 0.8098  valLogLoss: 0.4831
valf1: 0.8199  valLogLoss: 0.9267
valf1: 0.7925  valLogLoss: 1.2028
valf1: 0.8589  valLogLoss: 1.0469
Fold2 Best f1: 0.861 Best logloss: 0.555 Best epoch:171
--------------total_iters:45
valf1: 0.8302  valLogLoss: 0.8840
valf1: 0.8375  valLogLoss: 1.4171
valf1: 0.7500  valLogLoss: 2.3256
valf1: 0.8101  valLogLoss: 1.9320
Fold3 Best f1: 0.861 Best logloss: 1.023 Best epoch:83
--------------total_iters:45
valf1: 0.7927  valLogLoss: 0.5031
valf1: 0.7950  valLogLoss: 0.6513
valf1: 0.8125  valLogLoss: 0.7518
valf1: 0.7831  valLogLoss: 0.9152
Fold4 Best f1: 0.834 Best logloss: 0.666 Best epoch:86
--------------total_iters:45
valf1: 0.7871  valLogLoss: 0.4865
valf1: 0.8153  valLogLoss: 0.7650
valf1: 0.8344  valLogL

In [87]:
from tqdm import tqdm
import os
def load_model(weight_path):
    print(weight_path)
    model = SeqNet()
    model.set_state_dict(paddle.load(weight_path))
    model.to(device)
    model.eval()
    return model

@paddle.no_grad()
def predict(test_loader):
    ret = 0
    for i, model in enumerate(model_list):
        print('----model ', i)
        pres_list = []
        for data in tqdm(test_loader):
            inputs, _a = data
            inputs = inputs.cuda()
            outputs = model(inputs)
            pres_list+=F.sigmoid(outputs).detach().cpu().numpy().tolist()
        ret += np.array(pres_list) / len(model_list)
    return list(map(lambda x: 1 if x > 0.5 else 0, ret))

In [88]:
# device=torch.device('cuda')
model_list=[]
for i in range(10):
    model_list.append(load_model(path+'model/fold_'+str(i+1)+'_best.pth'))
import os

sub = pd.read_csv(path+'answer.csv')
test_loader = DataLoader(
        myDataset(sub, if_train=False), 
        batch_size=64, shuffle=False, num_workers=16)
sub['tag'] = predict(test_loader)
sub.head()

./model/fold_1_best.pth
./model/fold_2_best.pth
./model/fold_3_best.pth
./model/fold_4_best.pth
./model/fold_5_best.pth
./model/fold_6_best.pth
./model/fold_7_best.pth
./model/fold_8_best.pth
./model/fold_9_best.pth
./model/fold_10_best.pth
----model  0


100%|██████████| 7/7 [00:02<00:00,  2.81it/s]


----model  1


100%|██████████| 7/7 [00:02<00:00,  2.90it/s]


----model  2


...

----model  8


100%|██████████| 7/7 [00:02<00:00,  2.86it/s]


----model  9


100%|██████████| 7/7 [00:02<00:00,  2.80it/s]


,name,tag
0,VAL0001,1
1,VAL0002,1
2,VAL0003,1
3,VAL0004,0
4,VAL0005,1


In [89]:
sub.to_csv(path+'answer.csv', index=False)
!rm -rf  answer.csv.zip
!zip answer.csv.zip answer.csv

  adding: answer.csv (deflated 80%)
